### Import libraries

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Customize 

In [ ]:
# Customize parameters
my_input_file="0302 input_cost_data.csv"
my_regression="0302 regression_results.csv"

#Define % change levels 
percent_changes = [-15, -10, -5, 0, 5, 10, 15]

# User-specified variable to analyze 
selected_driver = 'Labor_Hours'  

### Load database and model

In [ ]:
# Load dataset 
input_file = my_input_file
df = pd.read_csv(my_input_file)

# Load regression coefficients from Excel
coef_df = pd.read_csv(my_regression)
coefficients = dict(zip(coef_df['Variable'], coef_df['Coefficient']))

print("Coefficients loaded from Excel:")
print(coefficients)

### Build Correlation matrix

In [ ]:
# Define correlation matrix
correlation_matrix = df.corr()

# Get base mean values
driver_vars = [k for k in coefficients.keys() if k != 'const']
mean_values = df[driver_vars].mean()

### Run sensitivity 

In [ ]:
# Run sensitivity scenarios 
results = []
for pct in percent_changes:
    adjusted = mean_values.copy()
    driver_multiplier = 1 + (pct / 100)
    adjusted[selected_driver] *= driver_multiplier

    for other_var in adjusted.index:
        if other_var != selected_driver:
            corr = correlation_matrix.loc[selected_driver, other_var]
            adjusted[other_var] *= (1 + corr * (pct / 100))

    predicted_cost = coefficients['const']
    for var in adjusted.index:
        predicted_cost += coefficients[var] * adjusted[var]

    results.append({
        'Change (%)': pct,
        'Selected_Driver': selected_driver,
        **adjusted.to_dict(),
        'Predicted_Overhead_Cost': predicted_cost
    })


### Create output, plot data, save results

In [ ]:

# Output DataFrame 
output_df = pd.DataFrame(results)
base_cost = output_df[output_df['Change (%)'] == 0]['Predicted_Overhead_Cost'].values[0]
output_df['Cost_Difference'] = output_df['Predicted_Overhead_Cost'] - base_cost

# Save to CSV 
output_csv = f"0304 Sensitivity_{selected_driver}.csv"
output_df.to_csv(output_csv, index=False)
print(f"CSV file saved: {output_csv}")

# Plot the cost change 
plt.figure(figsize=(10, 6))
plt.plot(output_df['Change (%)'], output_df['Cost_Difference'], marker='o')
plt.axhline(0, linestyle='--', color='gray')
plt.title(f'Overhead Cost Sensitivity to Changes in {selected_driver}')
plt.xlabel(f'Change in {selected_driver} (%)')
plt.ylabel('Change in Overhead Cost')
plt.grid(True)
plt.tight_layout()
plot_file = f"0304 Sensitivity_plot_{selected_driver}.png"
plt.savefig(plot_file)
plt.show()


print(f"Plot saved: {plot_file}")

# Compute the slope and intercept of the simulated curve

x = output_df['Change (%)'].values
y = output_df['Cost_Difference'].values

# Perform linear regression:
slope, intercept = np.polyfit(x, y, 1)

print(f"Slope of the plotted line: {slope:.4f}")
print(f"Intercept of the plotted line: {intercept:.4f}")
